# XGBoost.Dask in many threads

Sometimes we want to train many large XGBoost models in parallel.  We do so in this example with ...

1.  The `xgboost.dask` project to do large training runs
2.  Optuna to do hyper-parameter-optimization
3.  A thread pool, to run many of these in parallel
4.  Coiled to launch Dask clusters (but you could swap in your favorite Dask deployment technology as you like)

Using `xgboost.dask` from many threads tooks a couple of small tweaks across projects.  This notebook resulted in the following PRs and issues:

-  https://github.com/dask/distributed/issues/7377
-  https://github.com/dask/dask/pull/9723
-  https://github.com/dask/distributed/pull/7369
-  https://github.com/dmlc/xgboost/pull/8558 (mostly cosmetic, not necessary)
-  Also something in Coiled to allow package_sync to be thread-safe, should be released by 2022-12-07

In [1]:
FILEPATH="s3://prefect-dask-examples/nyc-uber-lyft/feature_table_fixed_upper_bound.parquet"

In [2]:
import datetime
import threading
from concurrent.futures import ThreadPoolExecutor

from distributed import Client
import dask.dataframe as dd
from coiled import Cluster
import coiled

import optuna
from dask_ml.metrics import mean_squared_error as lazy_mse
import xgboost as xgb
from xgboost.dask import DaskDMatrix

from dask_ml.datasets import make_classification_df
from dask_ml.model_selection import train_test_split, KFold
from dask_ml.preprocessing import OneHotEncoder
import dask.array as da
import dask.dataframe as dd
from s3fs import S3FileSystem
from xgboost.core import XGBoostError
import numpy as np

import pandas as pd

In [3]:
import dask, coiled
print("coiled:", coiled.__version__)
print("dask:", dask.__version__)
print("dask.distributed:", dask.distributed.__version__)
print("optuna:", optuna.__version__)
print("xgboost:", xgb.__version__)
print("coiled:", coiled.__version__)

coiled: 0.2.58
dask: 2022.12.1
dask.distributed: 2022.12.1
optuna: 3.1.0.dev
xgboost: 1.7.2
coiled: 0.2.58


### Train Model

In [4]:
# Here we subset data for cross-validation

def _make_cv(df, num_folds):
    frac = [1 / num_folds]*num_folds
    splits = df.random_split(frac, shuffle=True)
    for i in range(num_folds):
        train = [splits[j] for j in range(num_folds) if j != i]
        test = splits[i]
        yield train, test


In [5]:
def train_model(trial_number, study_params, n_splits=5, cluster_name = None, teardown_cluster=False):
    if cluster_name is None:
        thread_id = threading.get_ident()
        cluster_name = "xgb-nyc-taxi-" + str(thread_id)

    cluster = coiled.Cluster(
        worker_vm_types=["m6i.4xlarge"],
        scheduler_vm_types=["m6i.2xlarge"],
        package_sync=True, # copy local packages,
        name=cluster_name,
        shutdown_on_close=False,  # reuse cluster across runs
        show_widget=False,
        n_workers=10,
        use_best_zone=True,
        account="dask-engineering",
        backend_options={"region": "us-east-2", "spot": True, "spot_on_demand_fallback": True}
        )

    print("starting run")
    with Client(cluster) as client:
        print(client.id)
        with client.as_current():
            
            # Load and pre-process the DataFrame
            ddf = dd.read_parquet(FILEPATH)
            ddf = ddf.repartition(partition_size="128MB").persist()
            categorical_vars = ddf.select_dtypes(include="category")
            ddf = ddf.categorize(columns=categorical_vars)
            

            val_scores = []

            
            for i, (train, test) in enumerate(_make_cv(ddf, n_splits)):
                print(f"Starting training run {i}")
                start = datetime.datetime.now()
                train = dd.concat(train)
                try:
                    assert all(train[c].cat.known for c in categorical_vars)
                    assert all(test[c].cat.known for c in categorical_vars)
                except Exception as e:
                    cluster.shutdown()
                    raise RuntimeError(f"Categorical_vars are not known")

                y_train = train['trip_time'].to_frame()
                y_train = y_train.astype(float).persist()
                X_train = train.drop(columns=['trip_time']).persist()

                # Make the training data
                y_test = test['trip_time'].to_frame().persist()
                y_test = y_test.astype(float).persist()
                X_test = test.drop(columns='trip_time').persist()

                try:
                    print("Make dtrain")
                    dtrain = DaskDMatrix(client, X_train, y_train, enable_categorical=True)

                    print("Make dtest")
                    dtest = DaskDMatrix(client, X_test, y_test, enable_categorical=True)

                    print("Training model")

                    model = xgb.dask.train(
                        client,
                        {
                            'verbosity': 2,
                            'tree_method': 'hist', 
                            "objective": "reg:squarederror",
                            **study_params
                        },
                        dtrain,
                        num_boost_round=4,
                        evals=[(dtrain, "train")],
                    )

                    print("Make predictions")
                    # It's faster to run the prediction directly on X_test DataFrame
                    # We also need to confirm that predictions on dtest when it
                    # contains categoricals performs as expected
                    predictions = xgb.dask.predict(client, model, X_test)

                    print("Score the model")
                    score = lazy_mse(y_test.astype("float32").to_dask_array(lengths=True).reshape(-1,), 
                                     predictions.to_dask_array(lengths=True), squared=False,
                                    )
                    print(f"rmse_score:  {score}")
                    val_scores.append(score)
                    print(f"val_scores:  {val_scores}")

                except XGBoostError as e:
                    print(f"Trial {i} failed with {e}")
        return np.mean(val_scores)


In [6]:
train_options = dict(
    n_splits = 5 
)

In [7]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 75, 125),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.6),
        'subsample': trial.suggest_float('subsample', 0, 1),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0, 1),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }
    rmse = train_model(
        trial_number=trial.number,
        study_params=params, 
        n_splits=train_options["n_splits"],
    )
    print(f"final mse:  {rmse}")
    return rmse

In [8]:
# create a single study

study = optuna.create_study(study_name="nyc-taxi-study")

executor = ThreadPoolExecutor(4)

futures = [
    executor.submit(study.optimize, objective, n_trials=3) for _ in range(20)
]

[I 2023-01-02 20:30:30,622] A new study created in memory with name: nyc-taxi-study


  Cloning https://github.com/optuna/optuna.git (to revision e8a010bb58aea943866e5f7addf0de953228de99) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-pkqzhdxt


  Running command git clone --filter=blob:none --quiet https://github.com/optuna/optuna.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-pkqzhdxt
  Running command git rev-parse -q --verify 'sha^e8a010bb58aea943866e5f7addf0de953228de99'
  Running command git fetch -q https://github.com/optuna/optuna.git e8a010bb58aea943866e5f7addf0de953228de99


  Resolved https://github.com/optuna/optuna.git to commit e8a010bb58aea943866e5f7addf0de953228de99
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for optuna: filename=optuna-3.1.0.dev0-py3-none-any.whl size=360987 sha256=5d89ef0b74f9b336d86f0b718acfae88cdf56ee1a5046c3c3b78eb2269a82e36
  Stored in directory: /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-ephem-wheel-cache-c1vxl3tg/wheels/51/b0/4b/c05d88297ef6716b5a87865bbbc77cf5b3aa7d959460a80772
Successfully built optuna
  Cloning https://github.com/optuna/optuna.git (to revision e8a010bb58aea943866e5f7addf0de953228de99) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-7c54crbl


  Running command git clone --filter=blob:none --quiet https://github.com/optuna/optuna.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-7c54crbl
  Running command git rev-parse -q --verify 'sha^e8a010bb58aea943866e5f7addf0de953228de99'
  Running command git fetch -q https://github.com/optuna/optuna.git e8a010bb58aea943866e5f7addf0de953228de99


...

rmse_score:  349.86993408203125
val_scores:  [349.085, 349.6649, 349.738, 349.64014, 349.86993]
final mse:  349.599609375
Starting training run 0
Make dtrain
starting run
Client-f44d83be-8b15-11ed-8ab7-1a6d039d6526
Starting training run 0
Make dtrain
Make dtest
Training model
Starting training run 0
Make dtrain
Make dtest
Training model
Make dtest
Training model
Make dtest
Training model
Make predictions
Score the model
Make predictions
Score the model
rmse_score:  416.4392395019531
val_scores:  [416.48654, 416.38553, 416.43924]
Starting training run 3
Make dtrain
rmse_score:  490.1156005859375
val_scores:  [490.1156]
Starting training run 1
Make dtrain
Make predictions
Score the model
Make dtest
Training model
Make predictions
Score the model
rmse_score:  352.33001708984375
val_scores:  [352.33002]
Starting training run 1
Make dtrain
Make dtest
Training model
rmse_score:  353.2980041503906
val_scores:  [353.298]
Starting training run 1
Make dtrain
Make dtest
Training model
Make dtest


[I 2023-01-02 21:27:52,284] Trial 21 finished with value: 416.40386962890625 and parameters: {'n_estimators': 75, 'learning_rate': 0.31868042599446467, 'subsample': 0.804643565876754, 'max_depth': 6, 'colsample_bytree': 0.9774410645294822, 'min_child_weight': 3, 'colsample_bynode': 0.6717755434327176, 'colsample_bylevel': 0.9951341424262625, 'reg_alpha': 0.3765997303439763, 'reg_lambda': 0.28914045625262763}. Best is trial 19 with value: 349.36517333984375.


final mse:  416.40386962890625
starting run
Client-9d71b442-8b16-11ed-8ab7-1a6d039d6526
Make predictions
Score the model
Starting training run 0
Make dtrain
rmse_score:  490.23150634765625
val_scores:  [490.1156, 492.79193, 490.2315]
Starting training run 3
Make dtrain
Make predictions
Score the model
Make predictions
Score the model
rmse_score:  353.21063232421875
val_scores:  [352.33002, 353.11905, 353.21063]
Starting training run 3
Make dtrain
rmse_score:  353.27337646484375
val_scores:  [353.298, 353.3491, 353.27338]
Starting training run 3
Make dtrain
Make dtest
Training model
Make dtest
Training model
Make dtest
Make dtest
Training model
Training model
Make predictions
Make predictions
Score the model
Score the model
Make predictions
Score the model
rmse_score:  428.3814697265625
val_scores:  [428.38147]
Starting training run 1
Make dtrain
rmse_score:  492.6895446777344
val_scores:  [490.1156, 492.79193, 490.2315, 492.68954]
Starting training run 4
Make dtrain
Make predictions
rm

[I 2023-01-02 21:32:07,750] Trial 22 finished with value: 491.1979064941406 and parameters: {'n_estimators': 114, 'learning_rate': 0.30700983352754896, 'subsample': 0.7820711114939567, 'max_depth': 3, 'colsample_bytree': 0.8968842821148204, 'min_child_weight': 3, 'colsample_bynode': 0.5018712359570427, 'colsample_bylevel': 0.8550690814181371, 'reg_alpha': 0.37919074761262717, 'reg_lambda': 0.5019291671722065}. Best is trial 19 with value: 349.36517333984375.


final mse:  491.1979064941406
starting run
Make predictions
Client-35610dd4-8b17-11ed-8ab7-1a6d039d6526
Score the model


In [9]:
futures

[<Future at 0x17fe19fd0 state=finished returned NoneType>,
 <Future at 0x17fe3b640 state=finished returned NoneType>,
 <Future at 0x17fe31310 state=finished returned NoneType>,
 <Future at 0x17fe4e850 state=finished returned NoneType>,
 <Future at 0x17fe6b580 state=finished raised AttributeError>,
 <Future at 0x17fe73580 state=finished returned NoneType>,
 <Future at 0x17fe73670 state=running>,
 <Future at 0x17fe73760 state=running>,
 <Future at 0x17fe73850 state=running>,
 <Future at 0x17fe73940 state=running>,
 <Future at 0x17fe73a30 state=pending>,
 <Future at 0x17fe73b20 state=pending>,
 <Future at 0x17fe73c10 state=pending>,
 <Future at 0x17fe73d00 state=pending>,
 <Future at 0x17fe73df0 state=pending>,
 <Future at 0x17fe73ee0 state=pending>,
 <Future at 0x17fe73fd0 state=pending>,
 <Future at 0x17fe77100 state=pending>,
 <Future at 0x17fe771f0 state=pending>,
 <Future at 0x17fe772e0 state=pending>]

[I 2023-01-02 21:32:18,920] Trial 24 finished with value: 353.31903076171875 and parameters: {'n_estimators': 114, 'learning_rate': 0.5971836137652087, 'subsample': 0.7660021686165911, 'max_depth': 3, 'colsample_bytree': 0.8381323163880818, 'min_child_weight': 3, 'colsample_bynode': 0.779075792509333, 'colsample_bylevel': 0.8095377405277713, 'reg_alpha': 0.417914034753841, 'reg_lambda': 0.5105683257390563}. Best is trial 19 with value: 349.36517333984375.


rmse_score:  353.33984375
val_scores:  [353.298, 353.3491, 353.27338, 353.33496, 353.33984]
final mse:  353.31903076171875
starting run
rmse_score:  428.38128662109375
val_scores:  [428.38147, 428.3813]
Starting training run 2
Make dtrain
Client-3c48dbf4-8b17-11ed-8ab7-1a6d039d6526
Make predictions
Score the model
Starting training run 0
Make dtrain
rmse_score:  352.180908203125
val_scores:  [352.33002, 353.11905, 353.21063, 352.34763, 352.1809]


[I 2023-01-02 21:32:46,272] Trial 23 finished with value: 352.63763427734375 and parameters: {'n_estimators': 114, 'learning_rate': 0.5941571109656975, 'subsample': 0.7938708117120348, 'max_depth': 6, 'colsample_bytree': 0.5822965888810061, 'min_child_weight': 3, 'colsample_bynode': 0.7643325918779996, 'colsample_bylevel': 0.8272833753693533, 'reg_alpha': 0.11793487073956084, 'reg_lambda': 0.5071619039627417}. Best is trial 19 with value: 349.36517333984375.


final mse:  352.63763427734375
Starting training run 0
Make dtrain
starting run
Client-4ca087fe-8b17-11ed-8ab7-1a6d039d6526
Make dtest
Training model
Starting training run 0
Make dtrain
Make dtest
Training model
Make dtest
Training model
Make dtest
Training model
Make predictions
Score the model
rmse_score:  428.467041015625
val_scores:  [428.38147, 428.3813, 428.46704]
Starting training run 3
Make dtrain
Make predictions
Score the model
Make dtest
Training model
Make predictions
Score the model
Make predictions
Score the model
rmse_score:  451.24737548828125
val_scores:  [451.24738]
Starting training run 1
Make dtrain
rmse_score:  359.6910705566406
val_scores:  [359.69107]
Starting training run 1
Make dtrain
rmse_score:  457.3029479980469
val_scores:  [457.30295]
Starting training run 1
Make dtrain
Make predictions
Score the model


...

rmse_score:  465.1383361816406
val_scores:  [413.42438, 465.1849, 467.56918, 431.29062, 465.13834]
final mse:  448.521484375


2023-01-02 21:56:15,241 - distributed.client - ERROR - 
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/distributed/comm/core.py", line 291, in connect
    comm = await asyncio.wait_for(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/tasks.py", line 479, in wait_for
    return fut.result()
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/distributed/comm/tcp.py", line 511, in connect
    convert_stream_closed_error(self, e)
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/distributed/comm/tcp.py", line 142, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc.__class__.__name__}: {exc}") from exc
distributed.comm.core.CommClose

  Cloning https://github.com/optuna/optuna.git (to revision e8a010bb58aea943866e5f7addf0de953228de99) to /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-0voz_fes


  Running command git clone --filter=blob:none --quiet https://github.com/optuna/optuna.git /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-req-build-0voz_fes
  Running command git rev-parse -q --verify 'sha^e8a010bb58aea943866e5f7addf0de953228de99'
  Running command git fetch -q https://github.com/optuna/optuna.git e8a010bb58aea943866e5f7addf0de953228de99


  Resolved https://github.com/optuna/optuna.git to commit e8a010bb58aea943866e5f7addf0de953228de99
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for optuna: filename=optuna-3.1.0.dev0-py3-none-any.whl size=360987 sha256=1eab71bbde30ab05de02d1ce6737c42cd8810509a2d2dca06dac4f48d7e62c45
  Stored in directory: /private/var/folders/b5/f_y899x168j7cs2m7szjld5c0000gn/T/pip-ephem-wheel-cache-yllfnhbg/wheels/51/b0/4b/c05d88297ef6716b5a87865bbbc77cf5b3aa7d959460a80772
Successfully built optuna


In [10]:
study.best_params

{'n_estimators': 111,
 'learning_rate': 0.5973213766032188,
 'subsample': 0.7937042358727737,
 'max_depth': 6,
 'colsample_bytree': 0.6256120671859474,
 'min_child_weight': 3,
 'colsample_bynode': 0.6719821020977526,
 'colsample_bylevel': 0.9848329120293066,
 'reg_alpha': 0.49601346192828766,
 'reg_lambda': 0.4768295671695967}

In [11]:
study.best_value

349.36517333984375

In [12]:
study.best_trial

FrozenTrial(number=19, state=TrialState.COMPLETE, values=[349.36517333984375], datetime_start=datetime.datetime(2023, 1, 2, 21, 12, 24, 891411), datetime_complete=datetime.datetime(2023, 1, 2, 21, 22, 45, 496896), params={'n_estimators': 111, 'learning_rate': 0.5973213766032188, 'subsample': 0.7937042358727737, 'max_depth': 6, 'colsample_bytree': 0.6256120671859474, 'min_child_weight': 3, 'colsample_bynode': 0.6719821020977526, 'colsample_bylevel': 0.9848329120293066, 'reg_alpha': 0.49601346192828766, 'reg_lambda': 0.4768295671695967}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=125, log=False, low=75, step=1), 'learning_rate': FloatDistribution(high=0.6, log=False, low=0.1, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'max_depth': IntDistribution(high=9, log=False, low=3, step=1), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'min_child_weight': Int

In [13]:
len(study.trials)

42

In [14]:
import joblib

In [15]:
joblib.dump(study, "data/second_study.pkl")

['data/second_study.pkl']

2023-01-02 21:58:49,428 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/distributed/comm/tcp.py", line 498, in connect
    stream = await self.client.connect(
  File "/Users/greghayes/Documents/projects/dask-xgboost-nyctaxi/.venv/lib/python3.9/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/tasks.py", line 490, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last

In [ ]:
f = futures[0].result()

In [ ]:
cluster.shutdown()